In [1]:
from openai import OpenAI
import os
from trulens_eval import TruLlama, FeedbackMode, Feedback, Tru
from trulens_eval.feedback import Groundedness
from trulens_eval import OpenAI as fOpenAI
from llama_index import StorageContext, load_index_from_storage
import streamlit as st
import numpy as np
import google.auth
import pandas as pd

In [2]:
# Authenticate with Google Cloud
key_path = st.secrets["JSON_PATH"]
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
credentials, project_id = google.auth.default()

# Authenticate with OpenAI
os.environ["OPENAI_API_KEY"] = st.secrets["OPEN_AI_API_KEY"]

In [3]:
# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="ask_priya_index")

# Load index from the storage context
index = load_index_from_storage(storage_context)

query_engine = index.as_query_engine()

In [4]:
provider = fOpenAI()
f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


In [5]:
context_selection = TruLlama.select_source_nodes().node.text
f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [6]:
grounded = Groundedness(groundedness_provider=provider)
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [8]:
tru = Tru()
tru.reset_database()
tru_recorder = TruLlama(
    query_engine,
    app_id="evaluator",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


In [9]:
# Load evaluation questions
eval_questions = []
with open('data/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [11]:
!pip install boto3 botocore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 27.0 MB/s eta 0:00:0000:0100:01
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 8.5 MB/s eta 0:00:00
  Using cached urllib3-2.0.7-py3-none-any.whl (124 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymilvus 2.3.4 requires grpcio<=1.58.0,>=1.49.1, but you have grpcio 1.60.0 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import pandas as pd

In [10]:
# Run evaluation engine on each eval question
for question in eval_questions:
    with tru_recorder as recording:
        query_engine.query(question)

boto3,botocore is/are required for using BedrockEndpoint. You should be able to install it/them with
	pip install boto3 botocore


In [13]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Context Relevance,Groundedness,Answer Relevance
0,"""Explain the process to apply for OPT for a college senior.""","""To apply for Optional Practical Training (OPT) as a college senior, the student should follow the following process:\n\n1. Meet with the Designated School Official (DSO): The student should schedule a meeting with their DSO to discuss their eligibility for OPT and to receive guidance on the application process.\n\n2. Submit Form I-765: The student needs to complete and submit Form I-765, Application for Employment Authorization, to the U.S. Citizenship and Immigration Services (USCIS). This form can be found on the USCIS website.\n\n3. Gather Required Documents: The student should gather the necessary supporting documents, which may include a copy of their passport, Form I-20, I-94 record, and any other documents specified by the USCIS.\n\n4. Pay the Application Fee: The student must pay the required application fee when submitting Form I-765. The fee amount can be found on the USCIS website.\n\n5. Submit the Application: The student should submit the completed Form I-765, along with the supporting documents and application fee, to the USCIS. The application can be submitted online or by mail, as specified by the USCIS.\n\n6. Receive the Employment Authorization Document (EAD): If the OPT application is approved, the student will receive an Employment Authorization Document (EAD) from the USCIS. This document serves as proof of the student's employment authorization.\n\n7. Begin OPT Employment: Once the student receives the EAD, they can begin working in a job related to their field of study. The OPT period typically lasts for up to 12 months, but certain STEM degree holders may be eligible for a 24-month extension.\n\nIt is important for the student to consult with their DSO throughout the application process to ensure they meet all the requirements and deadlines.""",0.75,0.177778,1.0
1,"""Explain the process to apply to H1B.""","""I'm sorry, but the given context information does not provide any information about the process to apply for an H-1B visa.""",0.00,0.000000,1.0
2,"""What are the differences between single-intent and dual-intent visas?""","""A single-intent visa is a type of visa that requires the bearer to have the sole intention of visiting the United States temporarily and returning to their home country when their authorized stay expires. On the other hand, a dual-intent visa allows the bearer to have both the intention of visiting the United States temporarily and the intention of potentially immigrating to the United States in the future. It is important to note that the intent to immigrate cannot be the sole basis for denying a nonimmigrant petition, application for initial admission, change of status, or extension of stay in the R classification. The Department of State is responsible for visa adjudication, while U.S. Customs and Border Protection immigration inspectors determine the admission, length of stay, and conditions of stay at a port of entry.""",0.65,1.000000,1.0
3,"""Can a persion on OPT travel outside the U.S. with an expired visa stamp?""","""No, a person on OPT cannot travel outside the U.S. with an expired visa stamp.""",0.80,0.000000,1.0
4,"""Who qualify for STEM OPT extension?""","""F-1 students who receive science, technology, engineering, and mathematics (STEM) degrees included on the STEM Designated Degree Program List, are employed by employers enrolled in and maintain good standing with E-Verify, and who have received an initial grant of post-completion OPT employment authorization related to such a degree, may qualify for a 24-month extension of such authorization.""",1.00,1.000000,1.0
5,"""When is the earliest a student can apply for OPT?""","""The earliest a student can apply for OPT is after their program end date.""",0.85,0.000000,1.0
6,"""How is H1B different from OPT?""","""H-1B and OPT (Optional Practical Training) are two different immigration sta

In [14]:
# View aggregated metrics for all eval questions
tru.get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
evaluator,0.729167,0.530212,0.996,3.56,0.001867


In [ ]:
# Open Streamlit dashboard
import nest_asyncio

nest_asyncio.apply()
tru.run_dashboard()